In [381]:
from constraint import Problem, AllDifferentConstraint
import traceback
import copy
from functools import cmp_to_key

from utils import get_solutions_for_one_variable, get_other_variables, apply_constraint, MAX_NUMBER, MIN_NUMBER, Histories

DEBUG = False

In [382]:
def check_constraint_validity(new_problem, peoples, var1, var2, operation, result, histories):

    apply_constraint(new_problem, var1, var2, operation, result)

    for people in get_other_variables(peoples):
        candidates = get_solutions_for_one_variable(new_problem, people, histories)
        if len(candidates) == 0:
            yes_no = input(f"No candidates found for {people} if using this cmd, continue with this ? y/N")
            if yes_no == "y":
                continue
            else:
                raise Exception(f"No candidates found for {people} if using this cmd")
        if len(candidates) == 1:
            print(f"One candidate left for {people} == {candidates}")


def add_command(problem: Problem, cmd: str, histories, peoples):
    # cmd sous la forme "VAR1 VAR2 OPERATION RESULT"
    parsed_cmd = cmd.upper().split()

    var1 = parsed_cmd[0]
    var2 = parsed_cmd[1]
    operation = parsed_cmd[2]
    result = int(parsed_cmd[3])

    new_variables = []
    current_variables = list(problem._variables.keys())
    if var1 not in current_variables:
        new_variables.append(var1)
    if var2 not in current_variables:
        new_variables.append(var2)

    new_problem = copy.deepcopy(problem)
    new_problem.addVariables(new_variables, range(MIN_NUMBER, MAX_NUMBER + 1))
    check_constraint_validity(new_problem, peoples, var1, var2, operation, result, histories)
    
    problem.addVariables(new_variables, range(MIN_NUMBER, MAX_NUMBER + 1))
    apply_constraint(problem, var1, var2, operation, result)

    return var1, var2, operation, result

def get_best_operation_between_two_variable(problem: Problem, var1: str, var2: str, histories: Histories, constraints: list[str]):
    operation = ""
    target = var1

    def can_use_operation(var1, var2, operation):
        in_histories = histories.has_already_used(var1,var2,operation)
        in_constraints = operation in constraints

        return not in_histories and not in_constraints
    
    
    
    # PRIORITY 0: to set boundaries if any known boundaries
    if histories.step == 1:
        operation = "ADD"
        if can_use_operation(var1, var2, operation):
            return operation, "(First step)", target
    
    var1_candidates = get_solutions_for_one_variable(problem, var1, histories)

    candidates = var1_candidates

    # PRIORITY 0: use add and be lucky enough to get direct addition
    if len(candidates) in (1,2):
        target = var2
        var2_candidates = get_solutions_for_one_variable(problem, var2, histories)
        
        # PRIORITY 0: already know number
        if len(var2_candidates) == 1:
            return "MUL", "(Already known)", target
        
        # Same spirit as the PRIORITY 0: to set boundaries if any known boundaries
        operation = "ADD"
        if can_use_operation(var1, var2, operation):
            return operation, "(Lucky Test)", target
        
        # Switch the candidates and evaluate 
        # which operation is best to reduce var2_candidates
        candidates = var2_candidates

        
    # PRIORITY 1: if candidates differ by last digits
    last_digits = set(c % 10 for c in candidates)
    if len(last_digits) > 1:
        operation = "MUL"
        if can_use_operation(var1, var2, operation):
            return operation, "(Last digits)", target
    
    # Verify because not working sometimes
    # PRIORITY 2: if candidates differ by tens digit
    tens_digits = set(c // 10 for c in candidates)
    if len(tens_digits) > 1:
        operation = "ZER"
        if can_use_operation(var1, var2, operation):
            return operation, "(Tens digits)", target
        
    # PRIORITY 3: when spread is large
    spread = max(candidates) - min(candidates)
    if spread > 20:
        operation = "DIV"
        if can_use_operation(var1, var2, operation):
            return operation, "(Large spread)", target
        
    
    # DEFAULT: to multiplication for small difference
    return "MUL", "(Default)", target

def compare_two_operations(operation1, operation2, histories: Histories):
    if histories.step == 1:
        # Doesn't matter enough
        return 0
    
    # This part make the guess worst, LOL
    # 
    # if "DIV" in (operation1["operation"], operation2["operation"]):
    #     # We prioritize the DIV operation cause it's rare in the current algo
    #     if operation1["operation"] == "DIV":
    #         # [op1,op2]
    #         return -1
    #     else:
    #         # [op2,op1]
    #         return 1

    if ("(Already known)" in (operation1["reason"],operation2["reason"] )):
        if (operation1["reason"] == "(Already known)"):
            # [op2,op1]
            return 1
        else:
            # [op1,op2]
            return -1

    operation1_used_once_or_twice = operation1["used"] in (1,2)
    operation2_used_once_or_twice = operation2["used"] in (1,2)
    if operation1_used_once_or_twice or operation2_used_once_or_twice:        
        # We prioritize the people that was used only once
        if operation1_used_once_or_twice:
            # [op1,op2]
            return -1
        else:
            # [op2,op1]
            return 1
        
    # if no know variable yet but already used 3 other variables
    # 3 = number max of possible question per user
    # 2 variably is not enough 
    # suggest the prioritized operation between the three
    
    apparition1 = histories.number_of_apparition(operation1["var"])
    apparition2 = histories.number_of_apparition(operation2["var"])

    # bigger value will be after lower value
    return apparition1 - apparition2 

def get_operation_alread_used_in_list_of_ordered_operation(list_of_ordered_operation, people):
    already_used_operations = []
    for ordered_operation in list_of_ordered_operation:
        for operation in ordered_operation:
            if operation["var"] == people:
                already_used_operations.append(operation["operation"])
    return already_used_operations

def get_best_operation_to_know_my_variable(problem: Problem, peoples, me: str, histories: Histories):

    other_peoples = get_other_variables(peoples, me)

    # First set of proposition
    list_of_ordered_operation = []

    number_of_available_operation = 4
    for _ in range(number_of_available_operation):
        operations = []
        for people in other_peoples:
            operations_already_used = get_operation_alread_used_in_list_of_ordered_operation(list_of_ordered_operation, people)
            operation, reason, target = get_best_operation_between_two_variable(problem, me, people, histories, operations_already_used)
            operations.append(
                {
                    "used": histories.number_of_apparition(people),
                    "var": people,
                    "operation": operation,
                    "reason": reason,
                    "target": target
                }
            )
        ordered_operation = sorted(operations, key=cmp_to_key(lambda op1,op2: compare_two_operations(op1, op2, histories)))
        list_of_ordered_operation.append(ordered_operation)
    
    return list_of_ordered_operation


In [383]:
me = "K"
peoples = [me, "L", "P", "A", "B", "C", "D", "E", "F", "G"]
# peoples = [me, "L", "P", "O"]
histories = Histories()

problem = Problem()
problem.addConstraint(AllDifferentConstraint())
problem.addVariable(me, range(MIN_NUMBER, MAX_NUMBER + 1))


In [426]:
# voir les résultat possible pour une varible
# changer la valeur de var

# take times if launched without any constraint
solutions = "par défaut"

if histories.step == 1:
    result = input("Il s'agit du première étape, afficher les solutions ? (Y/n)")
    if result == "y":
        solutions = []
        for p in peoples:
            solutions.append({
                "var": p,
                "candidates": get_solutions_for_one_variable(problem, p, histories)
            })
else:
    solutions = []
    for p in peoples:
        solutions.append({
            "var": p,
            "candidates": get_solutions_for_one_variable(problem, p, histories)
        })
    
solutions

[{'var': 'K', 'candidates': [64]},
 {'var': 'L', 'candidates': [23]},
 {'var': 'P', 'candidates': [89, 44, 84, 49]},
 {'var': 'A', 'candidates': [10]},
 {'var': 'B', 'candidates': [71]},
 {'var': 'C', 'candidates': range(1, 101)},
 {'var': 'D', 'candidates': range(1, 101)},
 {'var': 'E', 'candidates': range(1, 101)},
 {'var': 'F', 'candidates': range(1, 101)},
 {'var': 'G', 'candidates': range(1, 101)}]

In [449]:
operation_sets = get_best_operation_to_know_my_variable(problem, peoples, me, histories)

for idx, operation_set in enumerate(operation_sets):
    print(f"=====[choix numéro {idx} selon ordre]")
    for idx, operation in enumerate(operation_set):
        print(idx, "-", operation)

=====[choix numéro 0 selon ordre]
0 - {'used': 3, 'var': 'L', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'L'}
1 - {'used': 4, 'var': 'P', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'P'}
2 - {'used': 3, 'var': 'A', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'A'}
3 - {'used': 2, 'var': 'B', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'B'}
4 - {'used': 1, 'var': 'C', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'C'}
5 - {'used': 1, 'var': 'D', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'D'}
6 - {'used': 1, 'var': 'E', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'E'}
7 - {'used': 1, 'var': 'F', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'F'}
8 - {'used': 1, 'var': 'G', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'G'}
=====[choix numéro 1 selon ordre]
0 - {'used': 3, 'var': 'L', 'operation': 'MUL', 'reason': '(Already known)', 'target': 'L'}
1 - {'used':

In [450]:
# ecrire le commande sous la forme : 
cmd = input("> VAR1 VAR2 OPERATION RESULT")

if cmd:
    try:
        var1, var2, operation, result = add_command(problem, cmd, histories, peoples)
        histories.add(var1, var2, operation, result)
        print("nouvelle contrainte ajouté!")
    except Exception as e:
        print("erreur (vérifier vos inputs)", e)
        if DEBUG:
           traceback.print_exc()
else:
    print("cmd non valide, vérifier les entrées")

# Show histories of applied constraint
len(histories.compact()), histories.compact()

cmd non valide, vérifier les entrées


(17,
 ['K L ADD 87',
  'K L MUL 2',
  'K P MUL 6',
  'K P ZER 2',
  'K P DIV 1',
  'K L ZER 4',
  'K A MUL 0',
  'K A ZER 5',
  'K A DIV 6',
  'K B ADD 135',
  'K B MUL 4',
  'K C ADD 104',
  'K D ADD 66',
  'K E ADD 156',
  'K F ADD 91',
  'K G ADD 157',
  'K P ADD 153'])

In [451]:
# Show histories of applied constraint
len(histories.compact()), histories.compact()

(17,
 ['K L ADD 87',
  'K L MUL 2',
  'K P MUL 6',
  'K P ZER 2',
  'K P DIV 1',
  'K L ZER 4',
  'K A MUL 0',
  'K A ZER 5',
  'K A DIV 6',
  'K B ADD 135',
  'K B MUL 4',
  'K C ADD 104',
  'K D ADD 66',
  'K E ADD 156',
  'K F ADD 91',
  'K G ADD 157',
  'K P ADD 153'])


```
[{'var': 'K', 'candidates': [64, 67, 69, 62]},
 {'var': 'L', 'candidates': [32, 34, 37, 39]},
 {'var': 'P', 'candidates': [73, 74, 78, 79, 53, 54, 58, 59]},
 {'var': 'A', 'candidates': [20]},
 {'var': 'B', 'candidates': range(1, 101)},
 {'var': 'C', 'candidates': range(1, 101)},
 {'var': 'D', 'candidates': range(1, 101)},
 {'var': 'E', 'candidates': range(1, 101)},
 {'var': 'F', 'candidates': range(1, 101)},
 {'var': 'G', 'candidates': range(1, 101)}]

(9,
 ['K L ADD 101',
  'K L MUL 8',
  'K L ZER 3',
  'K P MUL 6',
  'K P ZER 1',
  'K P DIV 1',
  'K A MUL 0',
  'K A ZER 4',
  'K A DIV 3'])
```

```
(6,
 ['K L ADD 87',
  'K L MUL 2',
  'K P MUL 6',
  'K L ZER 4',
  'K P ZER 2',
  'K P DIV 1'])

[{'var': 'K', 'candidates': [64]},
 {'var': 'L', 'candidates': [37]},
 {'var': 'P', 'candidates': [79]},
 {'var': 'A', 'candidates': range(1, 101)},
 {'var': 'B', 'candidates': range(1, 101)},
 {'var': 'C', 'candidates': range(1, 101)},
 {'var': 'D', 'candidates': range(1, 101)},
 {'var': 'E', 'candidates': range(1, 101)},
 {'var': 'F', 'candidates': range(1, 101)},
 {'var': 'G', 'candidates': range(1, 101)}]

(8,
 ['K L ADD 101',
  'K L MUL 8',
  'K L ZER 3',
  'K P MUL 6',
  'K P ZER 1',
  'K P DIV 1',
  'K L DIV 1',
  'K P ADD 143'])
```

(7,
 ['K L ADD 132',
  'K L MUL 6',
  'K P ZER 1',
  'K P DIV 1',
  'K A ZER 1',
  'K A DIV 1',
  'K L ZER 4'])